In [ ]:
import os
import sys
import torch
from torch.utils.data import DataLoader
import create_paths
from create_paths import smoothen_paths
import matplotlib.pyplot as plt

Configure the evaluation run

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATA_DIR = 'inputs' # the root directory containing the processed input data
DATASET = None      # (String) The name of the dataset to load. Leave as None to use the same dataset that the model was trained on.
SUBSET = 'eval'     # The name of the subset to load from

RUN_NAME = ''
STEP = 0

BATCH_SIZE = 50
NUM_SAMPLES = 5     # the number of example outputs to generate

Load the network(s)

In [ ]:
model = torch.load(f'checkpoints/{RUN_NAME}/gen/step{STEP}.tar')
gen = model['gen']
if DATASET == None:
    DATASET = model['dataset']

Get the input set

In [ ]:
# Make sure inputs exist for the desired dataset
input_dir = os.path.join(os.getcwd(), DATA_DIR, DATASET, SUBSET)
if not os.path.isdir(input_dir):
    print(f'ERROR: {input_dir} is not a valid directory')
    sys.exit(1)

# Load all inputs
inputs = []
truth = os.scandir(input_dir)
for item in truth:
    true_item = torch.load(item)
    inputs = inputs.append(true_item)

dataloader = DataLoader(inputs, batch_size=BATCH_SIZE, shuffle = True, drop_last = True)

Get the resulting outputs

In [ ]:
outputs = torch.empty(None)
for batch_idx, item in enumerate(dataloader):
    item = item.to(DEVICE)

    # Generate noise
    batch = torch.rand_like(item[:,0,:,:])
    batch = torch.cat((batch,item[:,1:,:,:]), axis=1)

    # Retrieve the batch of outputs
    item_out = item[:,-1,:,:]
    item_out = torch.cat((item_out, item[:,-2,:,:]), axis=1)
    item_out = torch.cat((item_out, gen(batch)), axis=1)
    item_out = torch.cat((item_out, item[:,0,:,:]), axis=1)

    item_out = item_out.to('cpu')
    outputs = torch.cat((outputs, item_out), axis=0)

outputs = outputs[:NUM_SAMPLES,:,:,:]

round outputs using Rachael's code

Format and save input, raw output, and rounded path